In [ ]:
%cd drive/MyDrive/Complexity-Measure/

/content/drive/MyDrive/Complexity-Measure


In [ ]:
!pip install gower;

  Created wheel for gower: filename=gower-0.0.5-py3-none-any.whl size=4231 sha256=a6157b375d10cc80472ed22f97eb7186a795c3d5bca2aafb04cb74a0275623af
  Stored in directory: /root/.cache/pip/wheels/3e/f9/9a/67122a959a424e9cbb4557a8366c871a30e31cd75f0d003db4
Successfully built gower


In [ ]:
from tensorflow.keras.applications import VGG16, Xception, ResNet50V2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
import numpy as np
import cv2
from tensorflow.keras.datasets import mnist, cifar10
from measures import *
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA

In [ ]:
from keras import backend as K
K.clear_session()

In [ ]:
(x_train_mnist, y_train_mnist), (x_test, y_test) = mnist.load_data()
x_train_mnist.shape, y_train_mnist.shape

11493376/11490434 [==============================] - 0s 0us/step


((60000, 28, 28), (60000,))

In [ ]:
(x_train_cifar, y_train_cifar), (x_test, y_test) = cifar10.load_data()
x_train_cifar.shape, y_train_cifar.shape

170500096/170498071 [==============================] - 2s 0us/step


((50000, 32, 32, 3), (50000, 1))

In [ ]:
def one_to_three_dim(x_train, dim):
  img_lst = []
  for i in range(len(x_train)):
    img = cv2.resize(x_train[i], dim, interpolation = cv2.INTER_AREA) 
    img = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    img_lst.append(img)
  rgb_arr = np.stack([img_lst],axis=4)
  rgb_arr_to_3d = np.squeeze(rgb_arr, axis=4)
  return rgb_arr_to_3d

In [ ]:
def feature_extractor(images, model, x, y, z):
  feature_list = []
  for i in range(len(images)):
    img_array = np.expand_dims(images[i], axis=0)
    features = model.predict(img_array)
    feature_list.append(features)
  feat_lt = np.reshape(feature_list, (-1, x*y*z))
  return feat_lt

In [ ]:
dim = (32, 32)
rgb_list = one_to_three_dim(x_train_mnist, dim)
print(rgb_list.shape)

(60000, 32, 32, 3)


In [ ]:
vgg = VGG16(weights='imagenet',
            include_top=False,
            input_shape=(32, 32, 3))

for layers in vgg.layers:
  layers.trainable=False

print(vgg.output)


58892288/58889256 [==============================] - 1s 0us/step
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 1, 512), dtype=tf.float32, name=None), name='block5_pool/MaxPool:0', description="created by layer 'block5_pool'")


In [ ]:
vgg_features_mnist = feature_extractor(rgb_list[0:10000], vgg, 1, 1, 512)
np.save("vgg16_features_mnist.npy", vgg_features_mnist)
vgg_features_cifar = feature_extractor(x_train_cifar[0:10000], vgg, 1, 1, 512)
np.save("vgg16_features_cifar.npy", vgg_features_cifar)

In [ ]:
PCA_X_mnist = x_train_mnist[0:10000]
PCA_Y_mnist = y_train_mnist[0:10000]
PCA_X_mnist = PCA_X_mnist/255
X_mnist_flat = PCA_X_mnist.reshape(-1, 28*28)
pca_mnist = PCA(0.9)
principalComponents_mnist = pca_mnist.fit_transform(X_mnist_flat)
principalComponents_mnist.shape

(10000, 85)

In [ ]:
compute_all_measures(principalComponents_mnist, PCA_Y_mnist)

F1 score:  0.3291913251953884
F1v score:  0.009387770114908826
F2 score:  1.7578469154652546e-07
F3 score:  3.7027312477213457








F4 score:  0.15668849313386993
R1 score:  0.0036440538195428363
R2 score:  0.0272
R3 score:  0.0129
N1 score:  0.1412
N2 score:  0.4292963703427572
N3 score:  0.07110000000000005
N4 score:  0.015000000000000013
LSC score:  0.99540873
Density score:  0.9233215921592159
ClsCoef score:  0.9202931282054626
Hubs score:  0.9941303588420946
T2 score:  0.0085
T3 score:  1.0493827160493827
T4 score:  117.6470588235294
C1 score:  0.0009846166111157073
C2 score:  0.0005563602001831258


In [ ]:
PCA_X_cifar = x_train_cifar[0:10000]
PCA_Y_cifar = y_train_cifar[0:10000].reshape(10000)
PCA_X_cifar = PCA_X_cifar/255
X_cifar_flat = PCA_X_cifar.reshape(-1, 32*32*3)
pca_cifar = PCA(0.9)
principalComponents_cifar = pca_cifar.fit_transform(X_cifar_flat)
principalComponents_cifar.shape

((10000, 96), array([6, 9, 9, ..., 1, 1, 5], dtype=uint8))

In [ ]:
compute_all_measures(principalComponents_cifar, PCA_Y_cifar)

F1 score:  0.8593337119300521
F1v score:  0.04442644318520849
F2 score:  3.00057906810976e-08
F3 score:  4.902795564973681
F4 score:  0.6822646324459863
R1 score:  0.17487009631787248
R2 score:  0.5517
R3 score:  0.472
N1 score:  0.7992
N2 score:  0.507231135804327
N3 score:  0.7303999999999999
N4 score:  0.24439999999999995
LSC score:  0.99992041
Density score:  0.9047292929292929
ClsCoef score:  0.904920420662385
Hubs score:  0.9905723421217026
T2 score:  0.0096
T3 score:  1.0434782608695652
T4 score:  104.16666666666667
C1 score:  0.00017168016136659414
C2 score:  9.649163956748641e-05


In [ ]:
FTR_X_mnist = np.load("vgg16_features.npy")
Y_mnist = y_train_cifar[0:1000]

In [ ]:
compute_all_measures(FTR_X_mnist, Y_mnist)

MNIST(PCA) F1 score:  0.3622180935427966


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/content/drive/MyDrive/Complexity-Measure/measures.py:159: RuntimeWarning: invalid value encountered in true_divide
  ratio = overlap_/range_


MNIST(PCA) F1v score:  nan
MNIST(PCA) F2 score:  0.0
MNIST(PCA) F3 score:  0.0


MNIST(PCA) F4 score:  0.0
MNIST(PCA) R1 score:  0.0
MNIST(PCA) R2 score:  0.0
MNIST(PCA) R3 score:  0.0
MNIST(PCA) N1 score:  0.187
MNIST(PCA) N2 score:  0.4569347842733395
MNIST(PCA) N3 score:  0.10899999999999999
MNIST(PCA) N4 score:  0.0050000000000000044
MNIST(PCA) LSC score:  0.990399
MNIST(PCA) Density score:  0.900002002002002
MNIST(PCA) ClsCoef score:  0.9010702312042197
MNIST(PCA) Hubs score:  0.9743793037518604
MNIST(PCA) T2 score:  2.048
MNIST(PCA) T3 score:  4.266666666666667
MNIST(PCA) T4 score:  0.48828125
MNIST(PCA) C1 score:  0.001914079013448755
MNIST(PCA) C2 score:  0.001115691269140795


In [ ]:
FTR_X_cifar = np.load("vgg16_features.npy")
Y_cifar = y_train_cifar[0:1000]

In [ ]:
compute_all_measures(FTR_X_cifar, Y_cifar)